In [1]:
import pandas as pd
import numpy as np

### Load annotation file

In [42]:
#Load gff3
use_cols1 = [0,3,4,8]
df1 = pd.read_csv('ct_manu_gene.gff3',sep='\t',header=None,usecols=use_cols1,encoding="utf-8-sig")
df1.columns = ['gene','start','end','info']
df1['original_name'] = df1['info'].apply(lambda x: x.split(';')[-1].split('=')[-1])
df1.drop('info',axis=1,inplace=True)
df1

,gene,start,end,original_name
0,tig00000000,43327,82701,20281-Ct.00g000010
1,tig00000000,92730,150361,20281-Ct.00g000020
2,tig00000000,156651,158870,20281-Ct.00g000030
3,tig00000000,158921,159397,20281-Ct.00g000040
4,tig00000000,159552,160696,20281-Ct.00g000050
...,...,...,...,...
43900,tig00108041,23229,26330,20281-Ct.00g439060
43901,tig00108042,8670,9796,20281-Ct.00g439070
43902,tig00108042,14053,14259,20281-Ct.00g439080
43903,tig00108042,14297,19693,20281-Ct.00g439090


### Load vcf file

In [43]:
#Load vcf
use_cols2 = [0,1,7]
df2 = pd.read_csv('ct_ann.vcf',delimiter='\t',header=None,usecols=use_cols2,encoding="utf-8-sig")
df2.columns = ['gene','pos','annotation']
df2

,gene,pos,annotation
0,tig00000000,157455,ANN=A|missense_variant|MODERATE|Ct.00g000030|C...
1,tig00000000,157456,ANN=A|missense_variant|MODERATE|Ct.00g000030|C...
2,tig00000000,157459,ANN=A|missense_variant|MODERATE|Ct.00g000030|C...
3,tig00000000,157462,ANN=A|missense_variant|MODERATE|Ct.00g000030|C...
4,tig00000000,157465,ANN=T|missense_variant|MODERATE|Ct.00g000030|C...
...,...,...,...
457382,tig00108041,23309,ANN=C|missense_variant|MODERATE|Ct.00g439010|C...
457383,tig00108041,23310,ANN=G|synonymous_variant|LOW|Ct.00g439010|Ct.0...
457384,tig00108041,23313,ANN=A|synonymous_variant|LOW|Ct.00g439010|Ct.0...
457385,tig00108041,23316,ANN=A|missense_variant|MODERATE|Ct.00g439010|C...


### Getting an intermediate file with SNPs in the genic regions

In [86]:
#Load significant gene file
#use_cols = [0]  #this part depends on the format of input sig files

#df3 = pd.read_csv('Significant_CULTRF_2000static_vars_PhenoCol-08.assoc.txt_NamesOnly.txt',header=None,usecols=use_cols,sep=' ')
df3 = pd.read_csv('Significant_soil_rooting_conditionsstatic_vars_PhenoCol-38.assoc.txt_NamesOnly.txt',sep='\t')
df3 = pd.DataFrame(df3.index)
df3.columns = ['Gene']

#df3 = pd.read_csv('ctar_MODCF-cloud-cov_soft2FDR_2020-06-11.txt',delimiter='\t',encoding="utf-8-sig")
#print(df3.columns.tolist())
uniq_max = df3.Gene.unique()

## getting variants in gene regions

d = {} # dictionary to hold intersected genes with the position of SNPs
l = [] # list to hold significant SNPs found in the intersection of gff and vcf

for item in uniq_max:  # iterating through the unique elements in the significant gene list
    gene1 = item.split(':')[0]      # getting contig name
    pos1 = item.split(':')[1]
    
    if gene1 in list(df1['gene']):  #check if the contig name matches in gff
        df5 = df1[df1['gene']==gene1]  # create a new dataframe of gff with the selected gene 
        df5.reset_index(drop=True, inplace=True)  #drop old index. VERY IMPORTANT!!!
    
    else:
        continue
    
    for i in range(len(df5)):
        if df5['start'][i]-100 <= int(pos1) <= df5['end'][i]+100: #check if correlated position falls inside a feature
            if item not in l:  #ignoring already added genes
                l.append(item)
            gene2 = df5['gene'][i] + ':' + str(df5['start'][i]) + ':' + str(df5['end'][i])+'+'+df5['original_name'][i] #gene found 
            if gene2 not in d.keys():
                d[gene2] = [pos1]
            else:
                d[gene2].append(pos1)
        

sig_list = [] # hold matched significant SNPs
gen_list = [] # hold intersected genes
snp_list = [] # hold number of SNPs found within intersected gene

for i in l: 
    for j in list(d.keys()):
        gen1 = i.split(':')
        gen2 = j.split('+')[0].split(':')
        if gen1[0] == gen2[0] and int(gen2[1])-100 <= int(gen1[1]) <= int(gen2[2])+100:
            sig_list.append(i)
            gen_list.append(j)
            snp_list.append(len(d[j]))
            
elem = [sig_list,gen_list,snp_list]
df_temp_min = pd.DataFrame(elem).T
df_temp_min.columns = ['Significant SNPs','Genes','Number of SNPs in the gene']
df_temp_min['Genes with start-end positions'] = df_temp_min['Genes'].apply(lambda x: x.split('+')[0])
df_temp_min['Original_name'] = df_temp_min['Genes'].apply(lambda x: x.split('+')[1])
df_temp_min_copy = df_temp_min.copy()
df_temp_min_copy.drop('Genes',axis=1,inplace=True)
df_temp_min_copy = df_temp_min_copy[['Significant SNPs','Genes with start-end positions','Original_name','Number of SNPs in the gene']]
df_temp_min_copy.to_csv('aet.txt',sep='\t')  

### Getiing gene function from InterproScan file

In [46]:
# Name    IPR Term        IPR Description Source  Source Term     Source Description      GO
use_cols1 = ['Name','Source Description']

dff3 = pd.read_csv('interpro.tab',sep='\t',usecols=use_cols1)
dff3['new_name'] = dff3['Name'].apply(lambda x: x[6:-4])
dff3.dropna(inplace=True)
dff3.reset_index(drop=True, inplace=True)

use_cols11 = [0,3,4]
dfexon = pd.read_csv('exxon.gff3',sep='\t',header=None,usecols=use_cols11,encoding="utf-8-sig")
dfexon.head()

#dff3

,0,3,4
0,tig00000000,43327,43433
1,tig00000000,43488,43754
2,tig00000000,58736,58866
3,tig00000000,58898,59237
4,tig00000000,65151,65280


In [87]:
func = []
gene = []
di = {}
ds = {}
dic = {}

dff1 = pd.read_csv('aet.txt',sep='\t',index_col=0)

for gene1 in dff1['Original_name']:
    dff4 = dff3[dff3['new_name'] == gene1[6:]]
    dff4.reset_index(drop=True, inplace=True)  #drop old index. VERY IMPORTANT!!! 
    for i in range(len(dff4)):
        #snp = dff1['Significant SNPs'][i]
        if dff4['new_name'][i] == gene1[6:]:
            if gene1 not in di:
                if dff4['Source Description'][i] not in ['FAMILY NOT NAMED','SUBFAMILY NOT NAMED']:
                    di[gene1] = [dff4['Source Description'][i]]
            
            else:
                if dff4['Source Description'][i] not in di[gene1]:
                    if dff4['Source Description'][i] not in ['FAMILY NOT NAMED','SUBFAMILY NOT NAMED']:
                        di[gene1].append(dff4['Source Description'][i])
                else:
                    continue
                    
for k,v in di.items():
    dic[k] = [' / '.join(di[k])]
dic

entry = dic.keys()
entry

dfw = pd.DataFrame(dic).T

fun = []
for gene in df_temp_min_copy['Original_name']:
    if gene not in dfw.index:
        fun.append('Gene not found in interproscan')
        continue
    for index in dfw.index:
        if gene == index:
            fun.append(dfw[0][index])
            

df_temp_min_copy['Gene Function'] = fun
df_temp_min_copy = df_temp_min_copy[['Significant SNPs','Genes with start-end positions','Original_name','Gene Function','Number of SNPs in the gene']]
df_temp_min_copy.to_csv('result_wind.csv',header=True,sep=',',index=None)

df_ = pd.read_csv('result_wind.csv')
#df_.head()
df_['Annotation'] = 'not_in_exon'
df_['Amino acid change'] = np.nan
df_['Nucleotide change'] = np.nan
for i in range(len(df_)):
    contig = df_['Significant SNPs'][i].split(':')
    
    df6 = df2[df2['gene']==contig[0]]  # create a new dataframe of gff with the selected gene 
    df7 = dfexon[dfexon[0]==contig[0]]
    df6.reset_index(drop=True, inplace=True)
    df7.reset_index(drop=True, inplace=True)
    
    for j in range(len(df7)):
        if int(df7[3][j]) <= int(contig[1]) <= int(df7[4][j]):
            for k in range(len(df6)):
                if int(df6['pos'][k]) == int(contig[1]):
                    df_.loc[i,'Annotation'] = df6['annotation'][k].split('|')[1]
                    df_.loc[i,'Amino acid change'] = df6['annotation'][k].split('|')[10]
                    df_.loc[i,'Nucleotide change'] = df6['annotation'][k].split('|')[9]
        
df_.to_csv('result_soil_rooting_conditions.csv',header=True,sep=',',index=None)   

